# Study of the reliability of borrowers

Customer - credit department of the bank. It is necessary to figure out whether the marital status and the number of clients of the client affect the fact of repaying the loan on time. The input data from the bank - statistics on customer payments. The data includes the following information about each client: the number of children in the family, the total length of service, age, level of education, marital status, gender, type of employment, the availability of debt on loan repayment, monthly income and the purpose of obtaining a loan.

The results of the study will be taken into account when building a ** loan scoring model ** - a special system that evaluates the ability of a potential borrower to return the loan to the bank.

# Project plan

## Study of data set
## data reporting
    2.1. Production processing
    2.2. Data type replacement
    2.3. Duplicate processing
    2.4. Lemmatization
    2.5. Data categorization
## Establishment of dependencies between data categories
## General conclusion

## 1 Study of data set

In [2]:
# I import the Pandas library and get general information on the table
import pandas as pd
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# We will display a few lines of the table to see what the data themselves are
data.loc[8:13]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости
11,0,-792.701887,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
13,0,-1846.641941,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля


** Conclusion ** Each line of the table contains certain information about the customer of the bank. To solve the problem, the most important columns Children, Family Status, Debt are. In the beginning, you need to solve gaps with passes and duplicates.

## 2 data rating

### Pass processing

In [4]:
# We calculate the number of passes using the ISNULL method
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

** Conclusion ** Discovered in two columns - total work experience and monthly income. The reasons for the income can be their instability, a person may not be arranged, then his experience is not calculated and income is also unofficial. Perhaps a person belongs to an incapable population, a non -working student or, in principle, unemployed.
To fill in the passes in the column with a monthly income, we calculate the average level of income for each type of employment. 
Serving experience does not play a special role for answering posed questions, so we can leave gaps in this column.

In [5]:
# Let's see all unique types of employment
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Replace the passes in the Total_income column with an average income for each type of employment:

In [6]:
# We group data on the type of employment and fill in the passes in the column "Monthly income" with a median value from the corresponding group.
# We display part of the table and check that the passes are filled
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median())).round()   
data[['income_type', 'total_income']].head(15)

,income_type,total_income
0,сотрудник,253876.0
1,сотрудник,112080.0
2,сотрудник,145886.0
3,сотрудник,267629.0
4,пенсионер,158616.0
5,компаньон,255764.0
6,компаньон,240526.0
7,сотрудник,135824.0
8,сотрудник,95857.0
9,сотрудник,144426.0


### Data type replacement

Monthly income is represented by material numbers, it would be more convenient to see them integer.

In [7]:
data['total_income'] = data['total_income'].values.astype(int)

Also in the column with the number of children pops up "-1", it is clear that this cannot be, most likely it is a typo and a minus is superfluous. Replace "-1" with "1", just as "20" is replaced with "2", here is also most likely a typo and zero extra.

In [8]:
# We replace the typos with the desired values ​​by the replace and check using the Unique method, how many children are repeated in the data
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Duplicate processing

Looking for duplicates:

In [9]:
# We calculate the number of duplicates using the Duplicated method
data.duplicated().sum()

54

In [10]:
# remove duplicates and perform the check
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

At the beginning of the study, when output part of the table, we see that in the “Education” column a different register. Here is the data to a single type.

In [11]:
# Here is the data in the column with formation to the lower register
# Check the correctness of the execution of the first lines of the first lines
data['education'] = data['education'].str.lower()
data['education'].head(10)

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
5     высшее
6     высшее
7    среднее
8     высшее
9    среднее
Name: education, dtype: object

Again, check the presence of duplicates.

In [12]:
# Call the Duplicated method
data.duplicated().sum()

17

After changing the register, new duplicates appeared, remove them.

In [13]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

** The conclusion ** duplicates could appear as a result of some kind of technological failure or human factor, perhaps someone has submitted an application twice.

### lemmatization

In [14]:
# Let's see all the unique goals on the column for the purpose of the loan
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

It can be seen that many goals are duplicated, combine them in the category.

In [15]:
# To do this, first we will divide the strings into separate words in a column with goals using lemmatization
from pymystem3 import Mystem
m = Mystem()

def credit_purpose(row):
    lemmas = m.lemmatize(row['purpose'])
    return ''.join(lemmas)
data['credit_purpose'] = data.apply(credit_purpose, axis=1)
data['credit_purpose'].head(10)

0                 покупка жилье\n
1       приобретение автомобиль\n
2                 покупка жилье\n
3    дополнительный образование\n
4               сыграть свадьба\n
5                 покупка жилье\n
6              операция с жилье\n
7                   образование\n
8         на проведение свадьба\n
9       покупка жилье для семья\n
Name: credit_purpose, dtype: object

In [16]:
# We select several categories: real estate, car, education, so on, and according to this we will change the column with goals
def purpose_grouped(row):
    lem=row['credit_purpose']
    if 'Housing' in lem or 'real estate' in lem:
        return 'real estate'
    elif 'automobile' in lem:
        return 'automobile'
    elif 'education' in lem:
        return 'education'
    else:
        return 'Other'
data['credit_purpose'] = data.apply(purpose_grouped, axis=1)
print(data['credit_purpose'].head(10))

0    недвижимость
1      автомобиль
2    недвижимость
3     образование
4          прочее
5    недвижимость
6    недвижимость
7     образование
8          прочее
9    недвижимость
Name: credit_purpose, dtype: object


** Conclusion ** When studying the column with the goals of the loan, they noticed that the same goals have different formulations, we simplified the column, highlighting several main categories. With the help of lemmatization, we identified separate words in the cells of the column "goal", among them "markers" were allocated for each category, with the help of which we formed a new column with the goals of lending, in which only 4 unique values. I combined everything that is connected with real estate into the category of real estate - purchase, construction, repair, kakay other operations (it is a large category, it could be divided into commercial real estate, residential real estate and small real estate transactions (for example, repair), but I thought that this is not so important for a request for this study), everything that is connected with the car is connected, the purchase, purchase, purchase, purchase, purchase, purchase, purchase, purchase, purchase, purchase, purchase. The transaction, new or supported, in the “Education” category, all education got into - additional, higher, profile, and category “Other”, in our case only borrowers who wanted to hold a wedding.

### Categorization of Data

In the previous step, we received a certain dictionary containing categories of lending goals. We allocated real estate, an atom car, education, and so on (for example, a loan for a wedding).

** Conclusion ** The category of column with the goals of lending helped to make a compact table and get rid of duplicates.

## 3 Establishment of dependencies between data categories

Let us find out if there is a dependence between the presence of children and the return of the loan on time.

In [17]:
# Let's make a summary table with the number of children and the total number of borrowers with delay
data_children = data.pivot_table(index=['children'], values='debt', aggfunc='sum')
data_children

,debt
children,
0,1063
1,445
2,202
3,27
4,4
5,0


According to the result, the result can be said that the more children - the less the likelihood of a loan arrears, but this will be incorrect, so we calculate the ratio of the number of debtors to borrowers with each number of children and we will bring it as a percentage

In [18]:
# We consider the number of borrowers for each number of children
data_children['all'] = data.groupby('children')['debt'].count()
data_children

,debt,all
children,,
0,1063,14091
1,445,4855
2,202,2128
3,27,330
4,4,41
5,0,9


In [19]:
# We calculate the percentage of debtors for each number of children
data_children['percent_of_debt'] = ((data_children['debt'] / data_children['all']) * 100).round(1)
data_children

,debt,all,percent_of_debt
children,,,
0,1063,14091,7.5
1,445,4855,9.2
2,202,2128,9.5
3,27,330,8.2
4,4,41,9.8
5,0,9,0.0


** Conclusion ** on the obtained values, it can be noted that for borrowers without children the percentage of debt is less, the value for borrowers with 5 children is considered emissions. This is due to the fact that children are a source of unpretentized expenses.

We will find out if there is a dependence between the family position and the return of the loan on time.

In [20]:
# Let's look at unique family statuses
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [21]:
# Let's make a summary table with the total number of debtors for each family status
data_family_status = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
data_family_status

,debt
family_status,
Не женат / не замужем,274
в разводе,85
вдовец / вдова,63
гражданский брак,388
женат / замужем,931


In [22]:
# We calculate the total number of borrowers for each family status
data_family_status['all'] = data.groupby('family_status')['debt'].count()
data_family_status

,debt,all
family_status,,
Не женат / не замужем,274,2810
в разводе,85,1195
вдовец / вдова,63,959
гражданский брак,388,4151
женат / замужем,931,12339


In [23]:
# We calculate the percentage of debtors for each family status
data_family_status['percent_of_debt'] = ((data_family_status['debt'] / data_family_status['all']) * 100).round(1)
data_family_status

,debt,all,percent_of_debt
family_status,,,
Не женат / не замужем,274,2810,9.8
в разводе,85,1195,7.1
вдовец / вдова,63,959,6.6
гражданский брак,388,4151,9.3
женат / замужем,931,12339,7.5


** The conclusion ** on the issues obtained, it can be noted that the percentage of the probability of debt is higher in unmarried/unmarried and living by a civil marriage of borrowers. This can be overstated by the fact that

We will find out if there is a dependence between the income level and the return of the loan on time.

In [24]:
# Find the magnitude of the maximum income
data['total_income'].max()

2265604

In [25]:
# Find the amount of minimum income
data['total_income'].min()

20667

In [26]:
# Find the average income value
data['total_income'].mean()

165320.05243777385

We allocate 3 groups in terms of income: up to 100,000, from 100,000 to 500,000 and more than 500,000.

In [27]:
# We will write a function that will appropriate the category for each income value: “low”, “medium” or “high”
# We will display a few lines to check the result
def income_level_group(row):
    if row['total_income'] <= 100000:
        return 'short'
    elif row['total_income'] > 500000:
        return 'high'
    else:
        return 'average'
data['income_level'] = data.apply(income_level_group, axis=1)
data['income_level'].loc[47:51]

47    средний
48    средний
49     низкий
50     низкий
51    средний
Name: income_level, dtype: object

In [28]:
# We will compose a consolidated table by income levels and the total amount of the debtor for each of them
data_total_income = data.pivot_table(index=['income_level'], values='debt', aggfunc='sum')
data_total_income

,debt
income_level,
высокий,14
низкий,354
средний,1373


In [29]:
# We calculate the total number of borrowers with each income level
data_total_income['all'] = data.groupby('income_level')['debt'].count()
data_total_income

,debt,all
income_level,,
высокий,14,222
низкий,354,4463
средний,1373,16769


In [30]:
# We calculate the percentage of debtors with each income level
data_total_income['percent_of_debt'] = ((data_total_income['debt'] / data_total_income['all']) * 100).round(1)
data_total_income

,debt,all,percent_of_debt
income_level,,,
высокий,14,222,6.3
низкий,354,4463,7.9
средний,1373,16769,8.2


** Conclusion ** on the issues obtained shows that there is no obvious dependence between the income level and the ability to repay loans on time. It can be noted that a larger percentage of debtors among customers with average income. This is probably explained that low -income clients “do not swing” by large amounts, and customers with high income do not have financial problems and more often repay the loan on time.

Consider how different credit goals affect its return on time.

In [31]:
# We will compose a summary table for lending goals and the number of debtors
data_credit_purpose = data.pivot_table(index=['credit_purpose'], values='debt', aggfunc='sum')
data_credit_purpose

,debt
credit_purpose,
автомобиль,403
недвижимость,782
образование,370
прочее,186


In [32]:
# We calculate the number of borrowers for each lending goal
data_credit_purpose['all'] = data.groupby('credit_purpose')['debt'].count()
data_credit_purpose

,debt,all
credit_purpose,,
автомобиль,403,4306
недвижимость,782,10811
образование,370,4013
прочее,186,2324


In [33]:
# We calculate the percentage of debtors for each lending goal
data_credit_purpose['percent_of_debt'] = ((data_credit_purpose['debt'] / data_credit_purpose['all']) * 100).round(1)
data_credit_purpose

,debt,all,percent_of_debt
credit_purpose,,,
автомобиль,403,4306,9.4
недвижимость,782,10811,7.2
образование,370,4013,9.2
прочее,186,2324,8.0


** Conclusion ** We can conclude that people are suitable more consciously and responsibly to large bushuits, such as real estate, this explains the relatively low percentage of debt on these loans

## 4 Conclusions

We have analyzed statistics on the solvency of bank customers. 
- Passes and duplicates were discovered, from which we successfully got rid of, also found typos in a column with the number of children and corrected them, relying on life logic.
- They made a column with the goals of the loan by categories: real estate, car, education, etc..

Next, we assessed the data on some characteristics of borrowers and the availability of debt on previous loans, made the following conclusions:
- the presence of children increases the likelihood of debt;
- unmarried/unmarried and living in a civil marriage often do not return the loan on time;
- borrowers with high income less often become debtors;
- Real estate loans are less likely to the rest in the expired.

On the main issue from the customer, it can be said that, regardless of marital status, the presence of children among borrowers increases the risk of loan arrears, the opposite, regardless of the number of children from customers or previously former in official family relations, less often occurs debts on a loan.

I think these two categories are not enough to assess the risk of debt. The analysis shows that the income level greatly affects the occurrence of debt (for example, if the client has high income and 5 children, this does not increase the risk of debt as low and unstable income from the borrower). Also, the purpose of lending is also important, we noted that real estate loans are less likely to happen, for example, their own housing is a more important and responsible aspect of life than a car. These categories should also be taken into account. Also, we rely on the presence of debt in the past everywhere, but the client can take a loan for the first time, respectively, there was no debt, this information is not enough, it would be also nice to take into account.